<a href="https://colab.research.google.com/github/subathra-adithan/FineRadScore/blob/main/MRI_date_fixation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you are asking another radiology resident to do it,
1. share access to the code
2. share access to the date folder
3. share protect range access in the two worklists
4. remind them to create a shortcut to date in their MyDrive


Download from LIS, name the file lisdate. No special characters in the name. Save as xlsx file. **Open the file, make sure column Q shows todays date.** Close the file and drop it under "date"
***Ignore above if you ran the CT_date fixation before this.***

In this code, you must edit the file name in FOUR places. Change the names first and then run the code.

1. ris sheet
2. lis sheet
3. merged csv file
4. For radiology office output file

# **Run the code ONLY AFTER you finish above.**
After you finish all work, push all the files in date folder into "old" subfolder

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
import pandas as pd
import datetime
!pip install xlsxwriter

In [90]:
#If you are doing for the first time, make a short cut to RD folder in your MyDrive
!ls '/content/drive/MyDrive/date'

'Common mistakes in CT and MRI Worklist preparation.xlsx'
'CT_date fixation.ipynb'
 CT_merged05.csv
 CT_Worklist.gsheet
 fixed
 for_office
 lis05.xlsx
'MRI_date fixation.ipynb'
 MRI_merged05.csv
 MRI_Worklist.gsheet
 old


# **1.**

In [91]:
#CHANGE file_path HERE
file_path = '/content/drive/MyDrive/date/fixed/05July_05November.xlsx'
df = pd.read_excel(file_path)
#df = pd.read_csv(file_path)

In [92]:
df['DATE_FIXED'] = pd.to_datetime(df['PROC_SCHEDULE_START_DATE']).dt.date
df['TIME'] = pd.to_datetime(df['PROC_SCHEDULE_START_DATE']).dt.time

In [93]:
df_ris = df[['DATE_FIXED','PAT_LAST_NAME', 'PAT_FIRST_NAME', 'PATIENT_ID','PROC_NAME', 'MODALITY', 'ACCESSION_NO','WORKPLACE','APP_CREATE_DATE']]

In [94]:
df_ris_MRI = df_ris[df_ris['MODALITY'] == 'MR']

In [95]:
len(df_ris_MRI)

2107

RIS alone is ready. The following steps are to merge LIS and RIS

# **2.**

In [96]:
# Make sure you have lis.xlsx under date
file_path = '/content/drive/MyDrive/date/lis05.xlsx'
df_lis = pd.read_excel(file_path)

In [97]:
df_lis_MRI = df_lis[df_lis['Investigation'].str.contains('MR')]

In [99]:
df_lis_MRI.rename(columns={'ScheduledDate': 'LIS_DATE'}, inplace=True)

/tmp/ipython-input-99-1533870343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lis_MRI.rename(columns={'ScheduledDate': 'LIS_DATE'}, inplace=True)


In [100]:
df_lis_MRI.drop(columns=["Unnamed: 0"], inplace=True)
df_lis_MRI.rename(columns={'HospitalNo': 'PATIENT_ID'}, inplace=True)
df_lis_MRI.drop(columns=['Ward', 'Bed Type', 'Bed No.'], inplace=True)

/tmp/ipython-input-100-662468641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lis_MRI.drop(columns=["Unnamed: 0"], inplace=True)
/tmp/ipython-input-100-662468641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lis_MRI.rename(columns={'HospitalNo': 'PATIENT_ID'}, inplace=True)
/tmp/ipython-input-100-662468641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lis_MRI.drop(columns=['Ward', 'Bed Type', 'Bed No.'], inp

In [101]:
len(df_lis_MRI)

14

In [102]:
merged_df = pd.merge(df_ris_MRI, df_lis_MRI, on='PATIENT_ID', how='outer', indicator=True)

In [103]:
merged_df.rename(columns={"_merge": "LIS_or_RIS"}, inplace=True)

In [104]:
merged_df['LIS_or_RIS'] = merged_df['LIS_or_RIS'].replace('left_only', 'RIS')
merged_df['LIS_or_RIS'] = merged_df['LIS_or_RIS'].replace('right_only', 'LIS')

/tmp/ipython-input-104-1535282306.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  merged_df['LIS_or_RIS'] = merged_df['LIS_or_RIS'].replace('left_only', 'RIS')
/tmp/ipython-input-104-1535282306.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  merged_df['LIS_or_RIS'] = merged_df['LIS_or_RIS'].replace('right_only', 'LIS')


# **3.**

In [105]:
#Change file name with date of month
merged_df.to_csv('/content/drive/MyDrive/date/MRI_merged05.csv',index=False)

**After you run this, you will find "MRI_mergedDate.csv" in RD folder. Open "worklist" and delete its contents.**
*Open "MRI_mergedDate.csv" with google sheets, Control A, Control C and Control V to worklist "scheduled" Change the dates in the three CT subfolders.
*
Protect the scheduled subsheet


In [106]:
merged_df['DATE_FIXED'] = pd.to_datetime(merged_df['DATE_FIXED']).dt.strftime('%d/%m/%Y')
merged_df['LIS_DATE'] = pd.to_datetime(merged_df['LIS_DATE']).dt.strftime('%d/%m/%Y')

In [107]:
len(merged_df)

2108

LIS RIS DISCREPANCY

In [108]:
df_diff_dates = merged_df[merged_df['LIS_DATE'].notna()]
df_diff_dates = df_diff_dates[df_diff_dates['LIS_DATE'] != df_diff_dates['DATE_FIXED']]
print(len(df_diff_dates))
#so both will be present if they have different dates in LIS and RIS, usually when it was fixed more than once or on more than one day.

5


In [109]:
df_diff_dates['message'] = 'discrepancy'

In [110]:
merged_df['ScheduledByName'] = merged_df['ScheduledByName'].str[:12]

In [111]:
diff = df_diff_dates[['message','DATE_FIXED', 'LIS_DATE', 'WORKPLACE', 'PATIENT_ID', 'PatientName', 'IPD/OPD', 'LIS_or_RIS','PROC_NAME','ScheduledByName', 'PAT_LAST_NAME', 'PAT_FIRST_NAME', 'Clinical Indication','RequestId', 'PatientMobileNo','APP_CREATE_DATE']]

len(diff)

FOR RADIOLOGY OFFICE

In [29]:
today = datetime.date.today().strftime('%d/%m/%Y')

In [112]:
print(today)

05/07/2025


In [31]:
'''
import datetime

yesterday = (datetime.date.today() - datetime.timedelta(days=1)).strftime('%d/%m/%Y')
print(yesterday)
'''

05/07/2025


In [25]:
#merged_df['STATUS'] = ''
#merged_df['FOR_CONSULTANT'] = ''

In [113]:
merged_df['Clinical Indication'] = merged_df['Clinical Indication'].str[:15]

In [35]:
'''
df_SSB_MRI = merged_df[(merged_df['DATE_FIXED'] == today) & (merged_df['WORKPLACE'] == 'MR1-SSB')][['DATE_FIXED','LIS_DATE','PROC_NAME', 'PATIENT_ID', 'PAT_LAST_NAME', 'PAT_FIRST_NAME', 'STATUS', 'IPD/OPD','Clinical Indication','FOR_CONSULTANT']]
df_Main_MRI = merged_df[(merged_df['DATE_FIXED'] == today) & (merged_df['WORKPLACE'] == 'MR2-MB')][['DATE_FIXED','LIS_DATE','PROC_NAME', 'PATIENT_ID', 'PAT_LAST_NAME', 'PAT_FIRST_NAME', 'STATUS','IPD/OPD','Clinical Indication','FOR_CONSULTANT']]
df_3T_MRI = merged_df[(merged_df['DATE_FIXED'] == today) & (merged_df['WORKPLACE'] == 'MR3-SSBEXP')][['DATE_FIXED','LIS_DATE','PROC_NAME', 'PATIENT_ID', 'PAT_LAST_NAME', 'PAT_FIRST_NAME', 'STATUS','IPD/OPD','Clinical Indication','FOR_CONSULTANT']]
'''

"\ndf_SSB_MRI = merged_df[(merged_df['DATE_FIXED'] == today) & (merged_df['WORKPLACE'] == 'MR1-SSB')][['DATE_FIXED','LIS_DATE','PROC_NAME', 'PATIENT_ID', 'PAT_LAST_NAME', 'PAT_FIRST_NAME', 'STATUS', 'IPD/OPD','Clinical Indication','FOR_CONSULTANT']]\ndf_Main_MRI = merged_df[(merged_df['DATE_FIXED'] == today) & (merged_df['WORKPLACE'] == 'MR2-MB')][['DATE_FIXED','LIS_DATE','PROC_NAME', 'PATIENT_ID', 'PAT_LAST_NAME', 'PAT_FIRST_NAME', 'STATUS','IPD/OPD','Clinical Indication','FOR_CONSULTANT']]\ndf_3T_MRI = merged_df[(merged_df['DATE_FIXED'] == today) & (merged_df['WORKPLACE'] == 'MR3-SSBEXP')][['DATE_FIXED','LIS_DATE','PROC_NAME', 'PATIENT_ID', 'PAT_LAST_NAME', 'PAT_FIRST_NAME', 'STATUS','IPD/OPD','Clinical Indication','FOR_CONSULTANT']]\n"

In [114]:
merged_df['ScheduledByName'] = merged_df['ScheduledByName'].str[:12]

In [115]:
df_SSB_MRI = merged_df[(merged_df['DATE_FIXED'] == today) & (merged_df['WORKPLACE'] == 'MR1-SSB')][['WORKPLACE','DATE_FIXED','LIS_DATE','PROC_NAME', 'PATIENT_ID', 'PAT_LAST_NAME', 'PAT_FIRST_NAME','Clinical Indication','APP_CREATE_DATE','ScheduledByName']]
df_Main_MRI = merged_df[(merged_df['DATE_FIXED'] == today) & (merged_df['WORKPLACE'] == 'MR2-MB')][['WORKPLACE','DATE_FIXED','LIS_DATE','PROC_NAME', 'PATIENT_ID', 'PAT_LAST_NAME', 'PAT_FIRST_NAME', 'Clinical Indication','APP_CREATE_DATE','ScheduledByName']]
df_3T_MRI = merged_df[(merged_df['DATE_FIXED'] == today) & (merged_df['WORKPLACE'] == 'MR3-SSBEXP')][['WORKPLACE','DATE_FIXED','LIS_DATE','PROC_NAME', 'PATIENT_ID', 'PAT_LAST_NAME', 'PAT_FIRST_NAME', 'Clinical Indication','APP_CREATE_DATE','ScheduledByName']]

In [116]:
def unique (df):
  df_unique = df.drop_duplicates(subset=['PAT_LAST_NAME', 'PATIENT_ID'])
  return df_unique

In [117]:
df_SSB_MRI = unique (df_SSB_MRI)
df_Main_MRI = unique (df_Main_MRI)
df_3T_MRI = unique (df_3T_MRI)

In [118]:
def alphabetic (df):
  df = df.sort_values(by=['PAT_LAST_NAME'])
  return df

In [119]:
df_SSB_MRI = alphabetic(df_SSB_MRI)
df_Main_MRI = alphabetic (df_Main_MRI)
df_3T_MRI = alphabetic (df_3T_MRI)

In [120]:
df_SSB_MRI.reset_index(drop=True, inplace=True)
df_Main_MRI.reset_index(drop=True, inplace=True)
df_3T_MRI.reset_index(drop=True, inplace=True)

# **4.**


In [121]:
#Change date in file name #79
with pd.ExcelWriter('/content/drive/MyDrive/date/for_office/MRI_05July025.xlsx', engine='xlsxwriter') as writer:


    df_SSB_MRI.to_excel(writer, sheet_name='SSB_MRI')

    workbook  = writer.book
    worksheet1 = writer.sheets['SSB_MRI']

    worksheet1.set_column('A:A', 2)
    worksheet1.set_column('B:B', 8)
    worksheet1.set_column('C:D', 10)
    worksheet1.set_column('E:E', 14)
    worksheet1.set_column('F:F', 8)
    worksheet1.set_column('G:G', 14)
    worksheet1.set_column('H:I', 13)
    worksheet1.set_column('J:J', 10)
    worksheet1.set_column('K:K', 10)

    df_Main_MRI.to_excel(writer, sheet_name='Main_MRI')
    worksheet2 = writer.sheets['Main_MRI']


    worksheet2.set_column('A:A', 2)
    worksheet2.set_column('B:B', 8)
    worksheet2.set_column('C:D', 10)
    worksheet2.set_column('E:E', 14)
    worksheet2.set_column('F:F', 8)
    worksheet2.set_column('G:G', 14)
    worksheet2.set_column('H:I', 13)
    worksheet2.set_column('J:J', 10)
    worksheet2.set_column('K:K', 10)


    df_3T_MRI.to_excel(writer, sheet_name='3T_MRI')
    worksheet3 = writer.sheets['3T_MRI']

    worksheet3.set_column('A:A', 2)
    worksheet3.set_column('B:B', 7)
    worksheet3.set_column('C:D', 10)
    worksheet3.set_column('E:E', 14)
    worksheet3.set_column('F:F', 8)
    worksheet3.set_column('G:G', 14)
    worksheet3.set_column('H:I', 13)
    worksheet3.set_column('J:J', 10)
    worksheet3.set_column('K:K', 10)

    diff.to_excel(writer, sheet_name='LIS_RIS_discrepancy')
    worksheet4 = writer.sheets['LIS_RIS_discrepancy']

    worksheet4.set_column('A:A',8)
    worksheet4.set_column('B:B',12)
    worksheet4.set_column('C:C',12)
    worksheet4.set_column('D:D',12)
    worksheet4.set_column('E:E',12)
    worksheet4.set_column('F:F',12)
    worksheet4.set_column('G:G',12)
    worksheet4.set_column('H:H',2)
    worksheet4.set_column('I:J',8)
    worksheet4.set_column('K:L',12)
    worksheet4.set_column('M:M',10)
    worksheet4.set_column('N:N',15)
    worksheet4.set_column('O:O',10)
    worksheet4.set_column('P:R',12)




## Send a message that worklist has been updated to 1. RD group 2. Stenographer